In [86]:
import pandas as pd
import numpy as np 
from scipy import signal
import matplotlib.pyplot as plt

#savitzky golay filter in scipy
from scipy.signal import savgol_filter

In [128]:
def subcarrier_fusion_module(filepath):
    csi_data = pd.read_csv(filepath, sep = "\t", header = None)
    
    #normalize the data
    #normalized_data = csi_data / np.max(np.abs(csi_data))
    
    #find the correlation of the data 
    correlation_matrix = np.corrcoef(csi_data.T)
    
    #find the eigenvalues and the associated eigenvectors of the data using numpy 
    eigenvalues, eigenvectors = np.linalg.eig(correlation_matrix)
    
    #extract the principal component from the eigenvectors, keeping only the second and third component
    eigenvectors = eigenvectors[:,1:3]
    
    #store PC2 here
    PC2 = eigenvectors[:,:1]
    
    #store PC3 here
    PC3 = eigenvectors[:,1:]
    
    return eigenvectors

In [131]:
eigen_vectors = subcarrier_fusion_module('1_alldata/30_subcarriers_1m/16_160_sample_A.txt')

#store PC2 here
PC2 = eigen_vectors[:,:1]
   
#store PC3 here/
PC3 = eigen_vectors[:,1:]

In [90]:
def savitzky_golay_filter(data, window_length, polyorder):
    filtered_data = np.zeros_like(data)
    
    for i in range(data.shape[1]):
        filtered_data[:,1] = savgol_filter(data[:,1], window_length, polyorder)
        
    return filtered_data

In [98]:
filtered_data = savitzky_golay_filter(eigen_vectors,5,2)

In [119]:
#filtered_data

In [99]:
def savitzky_golay_filter(data, window_length, polyorder):
    # Check if the window length is odd and greater than the polynomial order
    if window_length % 2 == 0 or window_length <= polyorder:
        raise ValueError("Window length must be odd and greater than polynomial order.")

    # Compute the number of points on each side of the center
    half_window = window_length // 2

    # Initialize the filtered data array
    filtered_data = np.zeros_like(data)

    # Calculate the weighting coefficients for the Savitzky-Golay filter
    x = np.arange(-half_window, half_window + 1)
    a_matrix = np.vander(x, N=polyorder + 1, increasing=True)
    coefficients = np.linalg.pinv(a_matrix)[0]

    # Apply the filter to each column of the data
    for i in range(data.shape[1]):
        for j in range(half_window, data.shape[0] - half_window):
            # Extract the local window of data
            window = data[j - half_window: j + half_window + 1, i]
            
            # Apply the Savitzky-Golay filter
            filtered_data[j, i] = np.dot(coefficients, window)

    return filtered_data


In [112]:
filtered_data = savitzky_golay_filter(eigen_vectors, 3,2)

In [116]:
len(filtered_data)

60

In [122]:
def sliding_window_variance(data, window_size):
    variances = []
    for i in range(len(data) - window_size + 1):
        window_data = data[i:i + window_size]
        variance = np.var(window_data)
        variances.append(variance)
    return np.array(variances)

def sliding_window_mean(data, window_size):
    means = []
    for i in range(len(data) - window_size + 1):
        window_data = data[i:i + window_size]
        mean = np.mean(window_data)
        means.append(mean)
    return np.array(means)

def adaptive_activity_segmentation(data, p, t1, t2):
    window_size = 5  

    # Calculate variance in sliding windows
    variance_data = sliding_window_variance(data, window_size)

    # Calculate mean in sliding windows
    mean_data = sliding_window_mean(variance_data, window_size)

    # Sort the mean data and get the maximum value
    sorted_means = np.sort(mean_data)
    max_value = sorted_means[-1]

    # Calculate threshold value T
    threshold = p * max_value

    # Discard values less than the threshold
    filtered_means = mean_data[mean_data >= threshold]

    # Calculate normalized segment length t
    t_values = np.linspace(t1, t2, num=60) 

    # Find the optimal t that satisfies the condition
    min_segment_length = float('inf')
    optimal_t = None
    for t in t_values:
        segment_length = t * len(data)
        if t1 < t < t2 and segment_length < min_segment_length:
            min_segment_length = segment_length
            optimal_t = t

    # Extract activity segments based on filtered mean data
    activity_segments = []
    for i in range(len(filtered_means)):
        if filtered_means[i] >= optimal_t * len(data):
            start_time = i * window_size
            end_time = (i + 1) * window_size - 1
            activity_segments.append((start_time, end_time))

    return activity_segments

In [121]:
p_value = 0.5
t1_value = 0.3
t2_value = 0.5
activity_segments = adaptive_activity_segmentation(filtered_data, p_value, t1_value, t2_value)
print("Activity Segments:", activity_segments)

Activity Segments: []
